##  All tables

This code is largely based on the implementation provided by Enge et al. (2021), available at https://osf.io/34ry2/. We are deeply grateful for their dedication to open research.

> Enge, A., Abdel Rahman, R., & Skeide, M. A. (2021). A meta-analysis of fMRI studies of semantic cognition in children. NeuroImage, 241, 118436. https://doi.org/10.1016/j.neuroimage.2021.118436

In [1]:
# Import necessary modules
from os import makedirs, path
from atlasreader import get_statmap_info
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
from nilearn import image, plotting, reporting
from scipy import stats
from scipy.stats import pearsonr
import os, fnmatch
from glob import glob
from pathlib import Path
import re

# print the current working directory
# print(os.getcwd())

The Python package you are importing, AtlasReader, is licensed under the
BSD-3 license; however, the atlases it uses are separately licensed under more
restrictive frameworks.
By using AtlasReader, you agree to abide by the license terms of the
individual atlases. Information on these terms can be found online at:
https://github.com/miykael/atlasreader/tree/master/atlasreader/data



In [2]:
# define the function to create a combined cluster table
def combined_cluster_table(
    img_files_z=[],
    img_files_ale=[],
    stub_keys=[],
    stub_colname="Analysis",
    cluster_extent=200,
    atlas="aal",
    td_jar=None,
    output_file="",
):

    # Create output director
    output_dir = path.dirname(output_file)
    makedirs(output_dir, exist_ok=True)

    # Create a list of DataFrames with peak and cluster stats for each image
    df_tuples = [
        get_statmap_info(img_file, cluster_extent=cluster_extent, atlas=atlas, voxel_thresh=0)
        for img_file in img_files_z
    ]
    dfs = [
        pd.DataFrame(
            {
                "Cluster #": df_tuple[0]["cluster_id"],
                "Size (mm3)": df_tuple[0]["volume_mm"],
                "Cluster labels": df_tuple[0][atlas],
                "Mean z": df_tuple[0]["cluster_mean"],
                "Peak z": df_tuple[1]["peak_value"],
                "Peak X": df_tuple[1]["peak_x"],
                "Peak Y": df_tuple[1]["peak_y"],
                "Peak Z": df_tuple[1]["peak_z"],
                "Peak label": df_tuple[1][atlas],
            }
        )
        for df_tuple in df_tuples
    ]

    # Add ALE values if available
    if img_files_ale:
        df_tuples_ale = [
            get_statmap_info(img_file, cluster_extent=cluster_extent, atlas=atlas, voxel_thresh=0)
            if img_file
            else (
                pd.DataFrame({"cluster_mean": [float("nan")]}),
                pd.DataFrame({"peak_value": [float("nan")]}),
            )
            for img_file in img_files_ale
        ]
        dfs_ale = [
            pd.DataFrame(
                {
                    "Mean ALE": df_tuple[0]["cluster_mean"],
                    "Peak ALE": df_tuple[1]["peak_value"],
                }
            )
            for df_tuple in df_tuples_ale
        ]
        for df, df_ale in zip(dfs, dfs_ale):
            df.insert(4, column="Mean ALE", value=df_ale["Mean ALE"])
            df.insert(6, column="Peak ALE", value=df_ale["Peak ALE"])

    # Concatenate into one big DataFrame
    df = pd.concat(dfs, keys=stub_keys)

    # Reformat numerical columns
    df["Size (mm3)"] = df["Size (mm3)"].apply(lambda x: "{:,.0f}".format(x))
    cols_int = ["Cluster #", "Peak X", "Peak Y", "Peak Z"]
    df[cols_int] = df[cols_int].applymap(int)
    cols_2f = ["Mean z", "Peak z"]
    df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
    if img_files_ale:
        cols_3f = ["Mean ALE", "Peak ALE"]
        df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))
        df[cols_3f] = df[cols_3f].replace("nan", "")

    # Add the stub column
    df.index = df.index.set_names(stub_colname, level=0)
    df.reset_index(level=stub_colname, inplace=True)
    mask = df[stub_colname].duplicated()
    df.loc[mask.values, [stub_colname]] = ""

    # Save to CSV
    df.to_csv(output_file, sep="\t", index=False)

    return df

## Tables - ALE results

#### Table 1. Single ALE Analysis: Healthy Controls (Self > Other Contrast)


In [3]:
tab0 = combined_cluster_table(
    img_files_z=["../Output/1_ALE/control_minus_patient_plus_only_self_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["../Output/1_ALE/control_minus_patient_plus_only_self_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="../Output/4_Tables/tab_ale_control_minus_patients_plus_only_self.tsv"
)

display(tab0)

/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_62893/3205870908.py:69: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_i

,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,904,33.63% Precuneus_R; 33.63% no_label; 32.74% Ci...,2.65,0.012,2.65,0.011,14,-44,38,Precuneus_R


In [11]:
tab1 = combined_cluster_table(
    img_files_z=["../Output/1_ALE/control_all_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["../Output/1_ALE/control_all_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="../Output/4_Tables/tab_ale_control.tsv"
)

display(tab1)

/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_61862/3205870908.py:69: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_i

,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"16,416",29.87% Cingulate_Ant_L; 28.41% Frontal_Sup_Med...,3.72,0.022,3.72,0.050,0,48,8,Cingulate_Ant_L
1,,2,"4,800",41.83% Frontal_Inf_Orb_2_L; 27.67% OFCpost_L; ...,3.72,0.021,3.72,0.035,-38,24,-14,OFCpost_L
2,,3,"3,256",43.00% Precuneus_L; 30.71% Cingulate_Post_L; 1...,3.72,0.023,3.72,0.043,-2,-54,26,Cingulate_Post_L
3,,4,"1,328",61.45% Cingulate_Mid_L; 38.55% Cingulate_Mid_R,2.73,0.022,2.73,0.036,2,-16,38,Cingulate_Mid_R
4,,5,"1,096",64.23% Angular_L; 35.77% Temporal_Mid_L,2.27,0.022,2.27,0.035,-44,-58,24,Angular_L


In [ ]:
tab1 = combined_cluster_table(
    img_files_z=["../Output/1_ALE/control_all_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["../Output/1_ALE/control_all_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="../Output/4_Tables/tab_ale_control.tsv"
)

display(tab1)

#### Table 2. Single ALE Analysis: Psychiatric Patients (Self > Other Contrast)


In [ ]:
tab2 = combined_cluster_table(
    img_files_z=["../Output/1_ALE/patient_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["../Output/1_ALE/patient_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="../Output/4_Tables/tab_ale_patient.tsv"
)

display(tab2)

#### Table 3. Single ALE Analysis: Psychiatric Patients Minus Healthy Controls (Self > Other Contrast)



In [ ]:
tab3 = combined_cluster_table(
    img_files_z=["../Output/1_ALE/patient_minus_control_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["../Output/1_ALE/patient_minus_control_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="../Output/4_Tables/tab_ale_patient_minus_control.tsv"
)

display(tab3)

#### Table 4. Single ALE Analysis: Healthy Controls Minus Psychiatric Patients (Self > Other Contrast)



In [ ]:
tab4 = combined_cluster_table(
    img_files_z=["../Output/1_ALE/control_minus_patient_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["../Output/1_ALE/control_minus_patient_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="../Output/4_Tables/subtraction.tsv"
)

display(tab4)

#### Table 5. Subtraction Analysis:  (Self > Other Contrast)

In [ ]:
from nibabel import save

# Compute seperate difference maps for controls > patients and patients > controls
img_sub = image.load_img("../Output/2_Subtraction/Control_all_minus_patient_z_thresh.nii.gz")
img_control_gt_patient = image.math_img("np.where(img > 0, img, 0)", img=img_sub)
img_patient_gt_control = image.math_img("np.where(img < 0, img * -1, 0)", img=img_sub)
_ = save(img_control_gt_patient, "../Output/2_Subtraction/control_greater_patient_z_thresh.nii.gz")
_ = save(img_patient_gt_control,  "../Output/2_Subtraction/patient_greater_control_z_thresh.nii.gz")

In [3]:
# Create a combined cluster table for the subtraction analysis
tab_sub = combined_cluster_table(
    img_files_z=["../Output/2_Subtraction/Control_all_minus_patient_z_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=[None],
    atlas="aal",
    output_file="../Output/4_Tables/tab_subtraction.tsv"
)

display(tab_sub)


/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_48511/3205870908.py:69: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_int] = df[cols_int].applymap(int)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_48511/3205870908.py:71: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_48511/3205870908.py:74: FutureWarning: DataFrame.applymap has been deprecated. Use Data

,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"1,584",93.94% Frontal_Inf_Tri_R,-3.42,,-3.89,,52,28,16,Frontal_Inf_Tri_R


#### Table 6. Conjunction Analysis:  (Self > Other Contrast)

In [ ]:
tab6 = combined_cluster_table(
    img_files_z=["../Output/2_Subtraction/controls_conj_patients_z.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["../Output/2_Subtraction/controls_conj_patients_ale.nii.gz", None],
    atlas="aal",
    output_file="../Output/4_Tables/tab_conjunction.tsv"
)

display(tab6)

#### Table 7. Single Analysis: Schizophrenia(SZ) patients (Self > Other Contrast)

In [ ]:
tab7 = combined_cluster_table(
    img_files_z=["../Output/1_ALE/SZ_z_size_level_thresh.nii.gz"],
    cluster_extent=10,
    stub_keys=["Activation likelihood estimation"],
    stub_colname="Analysis",
    img_files_ale=["../Output/1_ALE/SZ_stat_size_thresh.nii.gz", None],
    atlas="aal",
    output_file="../Output/4_Tables/tab_SZ.tsv"
)

display(tab7)

/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_69439/891482580.py:69: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_in

,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,Activation likelihood estimation,1,"1,584",44.95% Precuneus_L; 29.80% Cingulate_Post_L; 1...,3.35,0.010,3.35,0.017,-2,-54,28,Cingulate_Post_L


## Tables - All fsn results

In [4]:
def extract_value_at_peaks(img_peaks, img_values, colname_value):
    img_peaks = image.load_img(img_peaks)
    df, _ = get_statmap_info(img_peaks, cluster_extent=0, atlas="aal")
    x, y, z = [df[col] for col in ["peak_x", "peak_y", "peak_z"]]
    affine = np.linalg.inv(img_peaks.affine)
    coords_ijk = image.coord_transform(x, y, z, affine=affine)
    coords_ijk = np.array(coords_ijk, dtype="int").T
    img_values = image.load_img(img_values)
    dat = img_values.get_fdata()
    peak_values = [dat[tuple(coords_ijk[ix])] for ix in range(len(coords_ijk))]
    df[colname_value] = peak_values
    return df

In [7]:
prefixes = ["control_minus_patient", "patient", "SZ"]
imgs_peaks = ["../Output/1_ALE/" + prefix + "_z_thresh.nii.gz" for prefix in prefixes]
imgs_values = [
    "../Output/3_FSN/" + prefix + "/" + prefix + "_mean_fsn.nii.gz"
    for prefix in prefixes
]
dfs_fsn = [
    extract_value_at_peaks(img_peaks, img_values, colname_value="fsn")
    for img_peaks, img_values in zip(imgs_peaks, imgs_values)
]


df_fsn_all = pd.concat(
    [df.assign(contrast=prefix) for df, prefix in zip(dfs_fsn, prefixes)],
    ignore_index=True
)

display(df_fsn_all)

df_fsn_all.to_csv("../Output/4_Tables/fsn_all_peaks_table.tsv", sep="\t", index=False)

/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of

,cluster_id,peak_x,peak_y,peak_z,cluster_mean,volume_mm,aal,fsn,contrast
0,1.0,14.0,-44.0,38.0,3.757702,1248.0,34.62% Precuneus_R; 33.33% Cingulate_Mid_R; 32...,4.8,control_minus_patient
1,1.0,2.0,-52.0,30.0,3.670323,1520.0,45.79% Precuneus_L; 28.42% Cingulate_Post_L; 1...,3.7,patient
2,2.0,-10.0,60.0,24.0,3.528744,1352.0,94.08% Frontal_Sup_Medial_L; 5.92% Frontal_Sup...,2.3,patient
3,1.0,2.0,-52.0,30.0,3.73618,1584.0,44.95% Precuneus_L; 29.80% Cingulate_Post_L; 1...,6.5,SZ
